In [1]:
import numpy as np
import pandas as pd
import emoji
import re
import os
import nltk
from emosent import get_emoji_sentiment_rank, get_emoji_sentiment_rank_multiple

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import lemmy



In [2]:
df_lookup = pd.read_excel('2_headword_headword_polarity.xlsx', header=None)
df_lookup.columns = ["word", "instances", "type", "idk", "sentiment", "similar"]

df_lookup_sorted = df_lookup

df_lookup_sorted['similar'] = df_lookup['similar'].str.split(';')

df_lookup_expanded = df_lookup_sorted.explode('similar')
df_lookup_expanded = df_lookup_expanded.rename(columns={'similar': 'similar_word'}).reset_index(drop=True)
df_lookup_expanded['similar_word'] = df_lookup_expanded['similar_word'].astype(str).str.strip().str.lower()
df_lookup_expanded = df_lookup_expanded.sort_values(by='similar_word')

df_init = pd.read_excel('Mappe1.xlsx')
df_init['Tokenized Tweet Length'] = df_init['TEXT'].apply(lambda x: len(nltk.word_tokenize(str.lower(x))))


In [3]:
# Takes string returns sentiment (int/None)
# def WordScore(word):
#     idx = df_lookup_expanded['similar_word'].searchsorted(word.lower())
#     if idx < len(df_lookup_expanded) and df_lookup_expanded.iloc[idx]['similar_word'] == word:
#         print(word)
#         return df_lookup_expanded.iloc[idx]['sentiment']
#     else: return CompoundWords(word)

# print(WordScore('superhelt'))
def WordScore(word):
    return LookupWord(word)

def LookupWord(word):
    idx = df_lookup_expanded['similar_word'].searchsorted(word.lower())
    if idx < len(df_lookup_expanded) and df_lookup_expanded.iloc[idx]['similar_word'] == word:
        return df_lookup_expanded.iloc[idx]['sentiment']
    return None

print(WordScore('vi'))

None


In [4]:
# def CompoundWords(word):
#     for i in range (1,len(word)):
#         fst = WordScore(word[:i])
#         snd = WordScore(word[i:])
#         print(word[:i], "og", word[i:] )
#         if fst is not None and snd is not None:
#             return (fst+snd) / 2
#         else: continue
#     return None

# print(CompoundWords("sut"))
def CompoundWords(word):
    for i in range(1, len(word)):
        fst = LookupWord(word[:i])
        snd = LookupWord(word[i:])
        print(word[:i], "og", word[i:])
        if fst is not None and snd is not None:
            return (fst + snd) / 2
    return None



In [5]:
# Takes string returns ensemble sentiment
def TweetScores(tweet, neg_lst = ['ikke'], intensity_lst = ['meget', 'især', 'bedste']):
    ens_sentiment = []
    mod = 1 
    tweet_tokenized = nltk.word_tokenize(str.lower(tweet))
    for word in tweet_tokenized:
        curr_sent = WordScore(word)
        ens_sentiment.append([word, curr_sent * mod if curr_sent is not None else curr_sent])

        if word in neg_lst:
            mod = -1
        elif word in intensity_lst:
            mod = 2
        else: mod = 1

    emoji_sent_lst = get_emoji_sentiment_rank_multiple(tweet)
    for emoji in emoji_sent_lst:
        ens_sentiment.append([emoji['emoji_sentiment_rank']['unicode_name'], emoji['emoji_sentiment_rank']['sentiment_score'] * 5])

    
    return ens_sentiment


print(TweetScores('"Ole er en god mand der holder sine løfter. Han finder sit arbejde meningsfuldt og elsker at finde løsninger til hverdagens puslespil. Han kan godt lide madonna og har madlyst om torsdagen. En kæk observant knejt siger mange '))
        


[['``', None], ['ole', None], ['er', None], ['en', None], ['god', 3], ['mand', None], ['der', None], ['holder', None], ['sine', None], ['løfter', 1], ['.', None], ['han', None], ['finder', None], ['sit', None], ['arbejde', None], ['meningsfuldt', 1], ['og', None], ['elsker', 1], ['at', None], ['finde', None], ['løsninger', 1], ['til', None], ['hverdagens', None], ['puslespil', None], ['.', None], ['han', None], ['kan', None], ['godt', 2], ['lide', None], ['madonna', 1], ['og', None], ['har', None], ['madlyst', 1], ['om', None], ['torsdagen', None], ['.', None], ['en', None], ['kæk', 1], ['observant', 1], ['knejt', None], ['siger', None], ['mange', None]]


In [6]:
def get_total_sentiment(tweet):
    total = 0
    count = 0
    for item in TweetScores(tweet):
        if isinstance(item, list) and item[1] is not None:
            total += item[1]
            count += 1
        elif isinstance(item, (int, float)):  # emoji score
            total += item
            count += 1
    return total, count

df_init[['Total Auto Sentiment', 'Token Count']] = df_init['TEXT'].apply(
    lambda x: pd.Series(get_total_sentiment(x))
)

df_init['Avg Auto Sentiment'] = df_init['Total Auto Sentiment'] / df_init['Token Count']



In [7]:
threshhold = df_lookup_expanded['sentiment'].quantile([0.25, 0.5, 0.75])
print(threshhold)

df_init['Quant Sentiment'] = df_init['Avg Auto Sentiment'].apply(
    lambda x: 'Positive' if x >= 1 else ('Negative' if x <= -2 else 'Neutral'))



0.25   -2.0
0.50   -1.0
0.75    1.0
Name: sentiment, dtype: float64
